In [29]:
from pydantic import BaseModel, validator, Field
from typing import Optional, Union
from datetime import datetime
import rdflib
from rdflib import Graph
from pyfuseki import FusekiUpdate, FusekiQuery
from rdflib import Namespace
from rdflib import URIRef
import json
from api.src.schemas.authorities.authority import Authority
from api.src.function.authorities.makeGraph import MakeGraph
from api.src.function.authorities.generateID import GenerateId
from datetime import date
from pysolr import Solr
from api.src.function.authorities.makeLabel import MakeLabel

In [6]:
id = GenerateId()
id

'bkau-1'

In [9]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

In [11]:
solr.delete(q="*:*", commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">196</int>\n</lst>\n</response>\n'

In [2]:
with open('exemples\personalName.json', encoding='utf-8') as file:
    reader = file.read()
    authority = json.loads(reader)
    file.close()
request = Authority(**authority)
request.dict()

{'type': 'PersonalName',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': datetime.date(2023, 4, 26),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': {'label': 'BiblioKeia Beta',
   'generationDate': '2023-04-26T16:01:40'},
  'identifiedBy': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
   'value': 'n80002329'},
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'PersonalName',
   'elementValue': {'value': 'Machado de Assis', 'lang': None}},
  {'type': 'DateNameElement',
   'elementValue': {'value': '1839-1908', 'lang': None}}],
 'fullerName': {'type': 'PersonalName',
  'elementValue': {'value': 'Joaquim Maria Machado', 'lang': None}},
 'birthDate': '1839-06-21',
 'birthPlace': {'value': 'http://id.loc.gov/authorities/names',
  'label': 'Rio de Janeiro',
  'bas

In [34]:
request.fullerName.elementValue.value

'Joaquim Maria Machado'

In [30]:
label = MakeLabel(request.elementList)
label

'Machado de Assis, 1839-1908'

In [40]:
request.birthPlace.label

'Rio de Janeiro'

In [43]:
doc = {
    'id': id,
    'type': request.type,
    "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
    "label": f'{MakeLabel(request.elementList)}'
}
if request.fullerName:
    doc['fullerName'] = request.fullerName.elementValue.value
if request.birthDate:
    doc['birthDate'] = request.birthDate
if request.birthPlace:
    doc['birthPlace'] = request.birthPlace.label
doc


{'id': 'bkau-1',
 'type': 'PersonalName',
 'creationDate': '2023-04-26',
 'label': 'Machado de Assis, 1839-1908',
 'fullerName': 'Joaquim Maria Machado',
 'birthDate': '1839-06-21',
 'birthPlace': 'Rio de Janeiro'}

In [44]:
response = solr.add([doc], commit=True)
response

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":151}}\n'

In [3]:
graph = MakeGraph(request,id)
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [4]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(graph)
response

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Encountered " "<" "< "" at line 19, column 15.\r\nWas expecting one of:\r\n    <IRIref> ...\r\n    <PNAME_NS> ...\r\n    <PNAME_LN> ...\r\n    <BLANK_NODE_LABEL> ...\r\n    <VAR1> ...\r\n    <VAR2> ...\r\n    <ANON> ...\r\n    \n'

In [19]:
g = Graph()
g.parse('http://id.loc.gov/authorities/names/n80002329.madsrdf.rdf', format='xml')
g.serialize('mads/machado.ttl', format='ttl')

<Graph identifier=N52ddc3914ce34664b366746ac6e0f1ba (<class 'rdflib.graph.Graph'>)>

In [4]:
class Uri(BaseModel):
    value: str
    label: str

class Label(BaseModel):
    value: str
    lang: Optional[str]

class Element(BaseModel):
    type: str
    elementValue: Label

class Variant(BaseModel):
    type: str
    # variantLabel: str
    elementList: list[Element]

In [18]:
class Authority(BaseModel):
    type: str
    recordChangeDate: datetime = Field(default=datetime.now())
    elementList: list[Element]
    fullerName: Optional[Element]
    birthDate: Optional[str]
    birthPlace: Optional[Label]
    deathDate: Optional[str]
    hasBroaderAuthority: Optional[list[Uri]]
    subjectOf: Optional[list[Uri]]
    contributionOf: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    isMemberOfMADSCollection: str



subjects

PersonalName

In [24]:
id = 'n50048460'
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            identifiers:local "{id}" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            ri:recordStatus "novo"^^xsd:string ] ; 
            madsrdf:authoritativeLabel "{MakeLabel(request.elementList)}" ;
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ;
            { f'madsrdf:fullerName [ a madsrdf:PersonalName ; rdfs:label "{request.fullerName.elementValue.value}" ] ;' if request.fullerName else ''}            
            { f'madsrdf:birthDate "{request.birthDate}"' if request.birthDate else ''}     
            { f'madsrdf:birthPlace <{request.birthPlace.value}>' if request.birthPlace else ''}
            { f'madsrdf:deathDate "{request.deathDate}"' if request.deathDate else ''}   
            {f'madsrdf:hasBroaderAuthority {", ".join([ f"<{i.value}>" for i in request.hasBroaderAuthority])} ;' if request.hasBroaderAuthority else ''}
            {f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            {f'madsrdf:hasExactExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasExactExternalAuthority])} ;' if request.hasExactExternalAuthority else ''}

            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .
            
        }} 
        }}"""
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX name: <https://bibliokeia.com/authorities/name/> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>     
    INSERT DATA {
        GRAPH <https://bibliokeia.com/authorities/PersonalName/n50048460>
        {
        <https://bibliokeia.com/authorities/PersonalName/n50048460> a madsrdf:Authority, 
            madsrdf:PersonalName ;
            identifiers:local "n50048460" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "2023-04-24 20:44:23.373511"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizati

GenreForm

In [201]:
with open('mads/GenreForm.json', encoding='utf-8') as file:
    reader = file.read()
    authority = json.loads(reader)
    file.close()
request = Authority(**authority)
request.dict()

{'type': 'GenreForm',
 'recordChangeDate': datetime.datetime(2023, 4, 20, 20, 59, 47, 910492),
 'authoritativeLabel': {'value': 'Fiction', 'lang': 'en'},
 'elementList': [{'type': 'GenreFormElement',
   'elementValue': {'value': 'Fiction', 'lang': 'en'}}],
 'isMemberOfMADSCollection': 'https://bibliokeia.com/authorities/GenreForm/',
 'hasBroaderAuthority': [{'value': 'http://id.loc.gov/authorities/genreForms/gf2014026415',
   'label': 'Literature'}],
 'subjectOf': None,
 'hasCloseExternalAuthority': [{'value': 'http://id.worldcat.org/fast/1423787',
   'label': 'Literature'}],
 'hasNarrowerAuthority': [{'value': 'http://id.loc.gov/authorities/genreForms/gf2014026217',
   'label': 'Literature'},
  {'value': 'http://id.loc.gov/authorities/genreForms/gf2014026220',
   'label': 'Literature'}],
 'hasVariant': [{'type': 'GenreForm',
   'variantLabel': 'Stories',
   'elementList': [{'type': 'GenreFormElement',
     'elementValue': {'value': 'Stories', 'lang': None}}]}]}

In [8]:
id = 'gf2014026339'
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            identifiers:local "{id}" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            ri:recordStatus "novo"^^xsd:string ] ; 
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ;
            {f'madsrdf:hasBroaderAuthority {", ".join([ f"<{i.value}>" for i in request.hasBroaderAuthority])} ;' if request.hasBroaderAuthority else ''}
            {f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
             {f'madsrdf:hasNarrowerAuthority {", ".join([ f"<{i.value}>" for i in request.hasNarrowerAuthority])} ;' if request.hasNarrowerAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .
            
        }} 
        }}"""
print(graph)

NameError: name 'request' is not defined